In [ ]:
import numpy as np
%autosave 600

In [ ]:
from scikits.odes.odeint import odeint

In [ ]:
help(odeint)

In [ ]:
#from scipy.integrate import solve_ivp

For the following the Cython module must be installed: pip3 install Cython

In [ ]:
%load_ext Cython

In [ ]:
def fun(t, y, out):
    mu = 50
    out[0] = y[1]
    out[1] =  mu * (1 - y[0] ** 2) * y[1] - y[0]

In [ ]:
def jac(t, y, out):
    mu = 50
    out[:,:] = [
        [0, 1],
        [-2 * mu * y[0] * y[1] - 1, mu * (1 - y[0] ** 2)]]

In [ ]:
%%cython
def fun_cython(double t, double[:] y, double [:] f):
    cdef double mu = 50
    f[0] = y[1]
    f[1] = mu * (1 - y[0] * y[0]) * y[1] - y[0]

In [ ]:
%%cython -I /home/benny/git/odes/scikits/odes/sundials/ -I /usr/local/lib/python3.5/dist-packages/scikits.odes-2.3.0.dev0-py3.5-linux-x86_64.egg/scikits/odes/sundials/
### UPDATE ABOVE -I flag to installed odes/sundials sources !!
import numpy as np
cimport numpy as np
from scikits.odes.sundials.cvode cimport CV_RhsFunction
    
#scikits.odes allows cython functions only if derived from correct class
cdef class RhsFunction(CV_RhsFunction):
    cpdef int evaluate(self, double t,
                       np.ndarray[double, ndim=1] y,
                       np.ndarray[double, ndim=1] f,
                       object userdata = None) except? -1:
        cdef double mu = 50
        f[0] = y[1]
        f[1] = mu * (1 - y[0] * y[0]) * y[1] - y[0]
        return 0

In [ ]:
y0 = [2, 0]
outtimes = np.linspace(0, 1000, 101)

In [ ]:
sol1 = odeint(fun, outtimes, y0, method='cvode')

In [ ]:
stats = %prun -r -q odeint(fun, outtimes, y0, method='cvode')

In [ ]:
stats.print_stats(20)

In [ ]:
sol2 = odeint(RhsFunction(), outtimes, y0, method='cvode')

In [ ]:
stats = %prun -r -q odeint(RhsFunction(), outtimes, y0, method='cvode')

In [ ]:
stats.print_stats(20)

In [ ]:
#verify the two solutions are equal
for t, u1, u2 in zip(sol1.values.t, sol1.values.y,  sol2.values.y):
    print('{0:>4.0f} {1:15.6g} {2:15.6g}'.format(t, u1[0], u2[0]) )